## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,khatanga,RU,71.9667,102.5000,-10.61,99.0,100.0,4.32,overcast clouds
1,1,ilulissat,GL,69.2167,-51.1000,33.80,86.0,90.0,10.36,snow
2,2,punta arenas,CL,-53.1500,-70.9167,46.40,71.0,0.0,14.97,clear sky
3,4,puerto ayora,EC,-0.7393,-90.3518,79.00,91.0,72.0,8.01,broken clouds
4,5,peniche,PT,39.3558,-9.3811,59.00,87.0,100.0,1.01,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_range_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                (city_data_df["Max Temp"] >= min_temp),:]

In [5]:
# 4a. Determine if there are any empty rows.
temp_range_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 9 to 678
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              95 non-null     int64  
 1   City                 95 non-null     object 
 2   Country              95 non-null     object 
 3   Lat                  95 non-null     float64
 4   Lng                  95 non-null     float64
 5   Max Temp             95 non-null     float64
 6   Humidity             95 non-null     float64
 7   Cloudiness           95 non-null     float64
 8   Wind Speed           95 non-null     float64
 9   Current Description  95 non-null     object 
dtypes: float64(6), int64(1), object(3)
memory usage: 8.2+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = temp_range_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,kavieng,PG,80.35,moderate rain,-2.5744,150.7967,
15,atuona,PF,81.10,clear sky,-9.8000,-139.0333,
19,lorengau,PG,83.53,overcast clouds,-2.0226,147.2712,
30,takoradi,GH,80.29,light rain,4.8845,-1.7554,
46,butaritari,KI,81.64,light rain,3.0707,172.7902,
54,san patricio,US,80.60,clear sky,28.0170,-97.5169,
64,saint george,US,82.00,clear sky,37.1041,-113.5841,
80,sola,VU,83.25,overcast clouds,-13.8833,167.5500,
100,georgetown,MY,80.01,few clouds,5.4112,100.3354,
101,vanimo,PG,82.65,overcast clouds,-2.6741,141.3028,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    r = requests.get(base_url,params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i,"Hotel Name"] = r["results"][0]["name"]
    except:
        print("Hotel not found...skipping")

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [12]:
# 7. Drop the rows where there is no Hotel Name.
indicies = hotel_df[hotel_df["Hotel Name"] ==""].index
hotel_df = hotel_df.drop(index=indicies)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,kavieng,PG,80.35,moderate rain,-2.5744,150.7967,Nusa Island Retreat
15,atuona,PF,81.10,clear sky,-9.8000,-139.0333,Villa Enata
19,lorengau,PG,83.53,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
30,takoradi,GH,80.29,light rain,4.8845,-1.7554,Raybow International Hotel
46,butaritari,KI,81.64,light rain,3.0707,172.7902,Isles Sunset Lodge
54,san patricio,US,80.60,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
64,saint george,US,82.00,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
80,sola,VU,83.25,overcast clouds,-13.8833,167.5500,Leumerus Bungalows
100,georgetown,MY,80.01,few clouds,5.4112,100.3354,Cititel Penang
101,vanimo,PG,82.65,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "./WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [1]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}<dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [16]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))